In [1]:
%%javascript
// 必须手动跑
var kernel = IPython.notebook.kernel;
var _the_name = window.document.getElementById("notebook_name").innerHTML;
kernel.execute(`notebook_id = '${_the_name}'`);

<IPython.core.display.Javascript object>

In [2]:
notebook_id

'i-want-to-be-top-5-v5_last_21_days'

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn
import xgboost
import tsfresh
import imblearn
import gc
import os
from tqdm.notebook import trange, tqdm
import pickle
import threading
from threading import Thread
import xgboost as xgb

from multiprocessing import Pool, cpu_count

# 超参数

In [4]:
hyper_parameters = {
    ## 数据处理
    "granularity": 10,  # 多少的interval 5分钟试一试?

    "default_fc_parameters": { # tsfresh的特征
        'sum_values': None,
        'median': None,
        'mean': None,
        'length': None,
        'standard_deviation': None,
        'variance': None,
        'maximum': None,
        'minimum': None,
        'abs_energy': None,
        'absolute_sum_of_changes': None,
        'cid_ce': [{"normalize": False}],
    },
    "fillna": 0,
    ## 训练相关
    "pos_threshold": 60, # 多少时间为label=1
    "kfold": 2

}

import json
import base64

exp_id = base64.b64encode(bytes(json.dumps(hyper_parameters), 'utf-8')).decode()
exp_id

'eyJncmFudWxhcml0eSI6IDEwLCAiZGVmYXVsdF9mY19wYXJhbWV0ZXJzIjogeyJzdW1fdmFsdWVzIjogbnVsbCwgIm1lZGlhbiI6IG51bGwsICJtZWFuIjogbnVsbCwgImxlbmd0aCI6IG51bGwsICJzdGFuZGFyZF9kZXZpYXRpb24iOiBudWxsLCAidmFyaWFuY2UiOiBudWxsLCAibWF4aW11bSI6IG51bGwsICJtaW5pbXVtIjogbnVsbCwgImFic19lbmVyZ3kiOiBudWxsLCAiYWJzb2x1dGVfc3VtX29mX2NoYW5nZXMiOiBudWxsLCAiY2lkX2NlIjogW3sibm9ybWFsaXplIjogZmFsc2V9XX0sICJmaWxsbmEiOiAwLCAicG9zX3RocmVzaG9sZCI6IDYwLCAia2ZvbGQiOiAyfQ=='

# 数据前处理

## 函数库

In [5]:
# 从EDA中获取, 可以大大减少数据的数量
garbage_serial_number_list = np.array([
       'server_36955', 'server_24596', 'server_45057', 'server_24620',
       'server_39690', 'server_58416', 'server_18468', 'server_49273',
       'server_24817', 'server_46547', 'server_38212', 'server_25942',
       'server_38', 'server_24591', 'server_52634', 'server_55846',
       'server_43261', 'server_37735', 'server_25474', 'server_30664',
       'server_38500', 'server_18444', 'server_56581', 'server_2044',
       'server_38321', 'server_43081', 'server_24632', 'server_59467',
       'server_18432', 'server_30928', 'server_24947', 'server_14165',
       'server_35047', 'server_34', 'server_37732', 'server_33877',
       'server_45855', 'server_33638', 'server_53677', 'server_13156',
       'server_24756', 'server_55924', 'server_27360', 'server_43106',
       'server_3367', 'server_39373', 'server_53122', 'server_2838',
       'server_8626', 'server_49263', 'server_57237', 'server_31334',
       'server_27461', 'server_754', 'server_51237', 'server_2213',
       'server_45174', 'server_21118', 'server_39072', 'server_12467',
       'server_43531', 'server_3041', 'server_50112', 'server_34181',
       'server_33932', 'server_27534', 'server_1', 'server_32717',
       'server_25928', 'server_57710', 'server_37152', 'server_55948',
       'server_18486', 'server_12922', 'server_12791', 'server_6677',
       'server_3944', 'server_1279', 'server_47280', 'server_52505',
       'server_15269', 'server_51943', 'server_10292', 'server_37676',
       'server_15412', 'server_55422', 'server_37832', 'server_36941',
       'server_1759', 'server_15730', 'server_55442', 'server_6277',
       'server_25913', 'server_12428', 'server_34714', 'server_37337',
       'server_1284', 'server_18455', 'server_43112', 'server_6281',
       'server_43100', 'server_23', 'server_37552', 'server_43671',
       'server_36947', 'server_50455', 'server_18469', 'server_14947',
       'server_43096', 'server_13893', 'server_55453', 'server_57970',
       'server_24931', 'server_37017', 'server_55444', 'server_58707',
       'server_6586', 'server_55897', 'server_50631', 'server_56576',
       'server_49519', 'server_55404', 'server_19921', 'server_57464',
       'server_180', 'server_30765', 'server_31043'
], dtype=object)


enum_mca_id = [
    'Z',
    'AP',
    'AF',
    'E',
    'CD',
    'BB',
    'C',
    'CC',
    'F',
    'G',
    'EE',
    'AA',
    'AE',
    'BC',
    'AZ',
    'DE',
    'FF'
]



### 预处理的函数

In [6]:
def give_grid(df):
    for n_grid in [10, 20, 30]:
        df[f'row_grid_{n_grid}'] = (df['row'] // ((2**17-1) / n_grid)).astype(int)
        df[f'col_grid_{n_grid}'] = (df['col'] // ((2**10-1) / n_grid)).astype(int)
    return df

    
def preprocess(df):
    df = df[~df.serial_number.isin(garbage_serial_number_list)]
    df['dt_collect_time'] = pd.to_datetime(df['collect_time'])
    df = df.sort_values("dt_collect_time")

    if 'mca_id' in df:
        df = pd.concat(
            [
                df.reset_index(drop=True),
                pd.DataFrame(
                    data=[
                        [1 if mca_id == v else 0 for mca_id in enum_mca_id]
                        for v in tqdm(df['mca_id'].values, desc="mca_id")
                    ],
                    columns=[
                        "mca_id_" + v for v in enum_mca_id
                    ]
                ).reset_index(drop=True)
            ],
            axis=1
        )
        df = df.drop(columns=['mca_id'])
        
    if 'row' in df and 'col' in df:
        # df_address_log
        df = give_grid(df)
    
    return df

### 计算标签的函数

In [7]:
def give_it_label(_df, df_failure):
    merged_df = pd.merge(
        _df, 
        df_failure[['serial_number', 'dt_failure_time']],
        on=['serial_number'],
        how='left'
    )

    merged_df['remain'] = 60 * 24 * 365 # 否则就取一年
    failure_indices = merged_df[
        ~np.isnan(merged_df.dt_failure_time)
    ].index
    tmp_df = merged_df.iloc[failure_indices]
    merged_df.loc[failure_indices, 'remain'] = (
        tmp_df['dt_failure_time'] - tmp_df['dt_collect_time']
    ).dt.total_seconds() // 60

    return merged_df.drop(columns=[
        'collect_time', 'dt_failure_time'
    ])

### 特征工程的函数

In [8]:
def feature_transform(args):
    
    _df, column_name = args
    
    _columns = [
        'dt_date', 'dt_hour', 'dt_minute', 
        'remain'
    ]
    _df_common = _df.tail(1)[_columns].reset_index().drop(
        columns=['index']
    )
    _drop_columns = [
        'granularity',
    ]
    
    # 应该只有一行
    _df_feature = tsfresh.extract_features(
        _df.drop(columns=_columns + _drop_columns),
        column_id="serial_number", 
        default_fc_parameters=hyper_parameters["default_fc_parameters"],
        disable_progressbar=True,
        n_jobs=0
    ).reset_index().rename(columns={'index': 'serial_number'})
    
    if column_name is None:
        return pd.concat([_df_common, _df_feature], axis=1)
    # 只有一行
    return pd.concat([_df_common, _df_feature], axis=1)[column_name].values[0]

### 直接得到可训练的数据

In [9]:
def get_all_features(df_trainable):
    # 确保训练时候的数据是一致的
    return [
        col for col in df_trainable if col not in [
            'dt_date', 'dt_hour', 'dt_minute', 'serial_number', 
            'remain', 'granularity'
        ]
    ]

def get_data_trainable(df_address_log, df_mce_log, df_kernel_log, df_failure):
    
    # 加上特征和granularity
    df_data_table = pd.concat([df_address_log, df_mce_log, df_kernel_log])
    df_data_table = df_data_table.sort_values("dt_collect_time").reset_index(drop=True)
    df_data_table = give_it_label(df_data_table, df_failure)
    
    df_data_table = df_data_table.fillna(hyper_parameters["fillna"])
    
    df_data_table['dt_date'] = df_data_table.dt_collect_time.dt.date
    df_data_table['dt_hour'] = df_data_table.dt_collect_time.dt.hour
    df_data_table['dt_minute'] = df_data_table.dt_collect_time.dt.minute
    df_data_table['granularity'] = (df_data_table['dt_date'].astype(str) + 
        "_" + df_data_table['dt_hour'].astype(str) + 
        "_" + (df_data_table['dt_minute'] // hyper_parameters['granularity']).astype(str))
    
    # 得到可以训练的数据
    df_data_table_trainable = df_data_table.drop(columns=[
        'dt_collect_time',
    ])

    exp_df = [v[1] for v in df_data_table_trainable.head(1).groupby([
        'serial_number', 'granularity'
    ], sort=False)][0]
    feature_columns = feature_transform((exp_df, None)).columns

    df_grouped = df_data_table_trainable.groupby([
        'serial_number', 'granularity'
    ], sort=False)


    with Pool(88) as p:
        df_trainable_list = list(tqdm(
            p.imap(
                feature_transform, 
                [(group, feature_columns) for name, group in df_grouped], 
                chunksize=10
            ), 
            total=len(df_grouped)
        ))
        
    df_trainable = pd.DataFrame(data=df_trainable_list, columns=feature_columns)
    
    return df_trainable

In [27]:
df_address_log_fuck = pd.read_csv("./tcdata/memory_sample_address_log_round1_a_train.csv")


In [28]:
df_address_log_fuck.collect_time

0           2019-01-01 19:50:32
1           2019-01-01 01:31:23
2           2019-01-01 00:18:15
3           2019-01-01 02:21:42
4           2019-01-01 05:09:55
                   ...         
23442746    2019-05-31 00:07:49
23442747    2019-05-30 23:21:33
23442748    2019-05-30 23:21:46
23442749    2019-05-30 23:45:01
23442750    2019-05-30 23:21:31
Name: collect_time, Length: 23442751, dtype: object

## 得到可训练数据

### 训练数据

In [10]:
df_address_log = pd.read_csv("./tcdata/memory_sample_address_log_round1_a_train.csv")
df_address_log = preprocess(df_address_log)

df_mce_log = pd.read_csv("./tcdata/memory_sample_mce_log_round1_a_train.csv")
df_mce_log = preprocess(df_mce_log)

df_kernel_log = pd.read_csv("./tcdata/memory_sample_kernel_log_round1_a_train.csv")
df_kernel_log = preprocess(df_kernel_log)

df_failure = pd.read_csv("./tcdata/memory_sample_failure_tag_round1_a_train.csv")
df_failure['dt_failure_time'] = pd.to_datetime(df_failure['failure_time'])
df_failure = df_failure.sort_values("dt_failure_time")



/home/tiger/anaconda3/envs/research/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


mca_id:   0%|          | 0/7225384 [00:00<?, ?it/s]

In [12]:
import datetime
sample_date = datetime.date(2019, 5, 1)

In [33]:
df_address_log_sample = df_address_log[
    (df_address_log.dt_collect_time.dt.date == sample_date) &
    (df_address_log.dt_collect_time.dt.hour == 3) &
    (df_address_log.dt_collect_time.dt.minute < 10)
]

df_mce_log_sample = df_mce_log[
    (df_mce_log.dt_collect_time.dt.date == sample_date) &
    (df_mce_log.dt_collect_time.dt.hour == 3)  &
    (df_mce_log.dt_collect_time.dt.minute < 10)
]

df_kernel_log_sample = df_kernel_log[
    (df_kernel_log.dt_collect_time.dt.date == sample_date)&
    (df_kernel_log.dt_collect_time.dt.hour == 3)  &
    (df_kernel_log.dt_collect_time.dt.minute < 10)
]

In [34]:
df_address_log_sample.shape[0] + df_mce_log_sample.shape[0] + df_kernel_log_sample.shape[0]

1921

In [41]:
df_concated_sample_new = pd.concat(
    [df_address_log_sample, 
     df_mce_log_sample,
     df_kernel_log_sample]
).reset_index(drop=True).sort_values("dt_collect_time").fillna(0).reset_index(drop=True)


df_concated_sample_new['dt_date'] = df_concated_sample_new.dt_collect_time.dt.date
df_concated_sample_new['dt_hour'] = df_concated_sample_new.dt_collect_time.dt.hour
df_concated_sample_new['dt_minute'] = df_concated_sample_new.dt_collect_time.dt.minute
df_concated_sample_new['granularity'] = (df_concated_sample_new['dt_date'].astype(str) + 
    "_" + df_concated_sample_new['dt_hour'].astype(str) + 
    "_" + (df_concated_sample_new['dt_minute'] // hyper_parameters['granularity']).astype(str))

In [43]:
with open('df_2019_5_1_third_hour.pickle', 'wb') as f:
    pickle.dump(df_concated_sample_new, f)

In [29]:
df_concated_sample_new = pd.concat(
    [df_address_log_sample.reset_index(drop=True), 
     df_mce_log_sample.reset_index(drop=True),
     df_kernel_log_sample.reset_index(drop=True)]
).reset_index(drop=True).sort_values("dt_collect_time").reset_index(drop=True)

In [30]:
df_concated_sample_old = pd.concat(
    [df_address_log_sample.reset_index(drop=True), 
     df_mce_log_sample.reset_index(drop=True),
     df_kernel_log_sample.reset_index(drop=True)]
).reset_index(drop=True).sort_values(["dt_collect_time", ""]).reset_index(drop=True)

In [22]:
df_concated_sample_old = pd.concat(
    [df_address_log_sample, 
     df_mce_log_sample,
     df_kernel_log_sample]
).sort_values("dt_collect_time").reset_index(drop=True)

In [32]:
df_concated_sample_new[
    ~df_concated_sample_new.isnull()
].values == df_concated_sample_old[
    ~df_concated_sample_new.isnull()
].values

array([[ True,  True,  True, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

In [26]:
df_concated_sample_new

,serial_number,memory,rankid,bankid,row,col,collect_time,manufacturer,vendor,dt_collect_time,...,3_hwerr_kp,1_hwerr_fl,3_hwerr_r,_hwerr_cd,3_sup_mce_note,3_cmci_sub,3_cmci_det,3_hwerr_pi,3_hwerr_o,3_hwerr_mce_l
0,server_6830,4.0,0.0,4.0,73055.0,672.0,2019-05-01 00:00:03,0.0,1.0,2019-05-01 00:00:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,server_6830,NaN,NaN,NaN,NaN,NaN,2019-05-01 00:00:03,0.0,1.0,2019-05-01 00:00:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,server_28400,NaN,NaN,NaN,NaN,NaN,2019-05-01 00:00:04,1.0,2.0,2019-05-01 00:00:04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,server_28400,NaN,NaN,NaN,NaN,NaN,2019-05-01 00:00:05,1.0,2.0,2019-05-01 00:00:05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,server_20000,NaN,NaN,NaN,NaN,NaN,2019-05-01 00:00:17,2.0,0.0,2019-05-01 00:00:17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79148,server_24831,19.0,0.0,8.0,111826.0,1008.0,2019-05-01 23:59:10,0.0,0.0,2019-05-01 23:59:10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79149,server_24831,NaN,NaN,NaN,NaN,NaN,2019-05-01 23:59:10,0.0,0.0,2019-05-01 23:59:10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79150,server_24831,NaN,NaN,NaN,NaN,NaN,2019-05-01 23:59:10,0.0,0.0,2019-05-01 23:59:10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79151,server_37091,22.0,1.0,6.0,73711.0,488.0,2019-05-01 23:59:56,2.0,0.0,2019-05-01 23:59:56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_trainable = get_data_trainable(df_address_log, df_mce_log, df_kernel_log, df_failure)

### Leakage数据

In [ ]:
df_address_log_leak = pd.read_csv("./tcdata/memory_sample_address_log_round1_b_test.csv")
df_address_log_leak = preprocess(df_address_log_leak)

df_mce_log_leak = pd.read_csv("./tcdata/memory_sample_mce_log_round1_b_test.csv")
df_mce_log_leak = preprocess(df_mce_log_leak)


df_kernel_log_leak = pd.read_csv("./tcdata/memory_sample_kernel_log_round1_b_test.csv")
df_failure_leak = df_kernel_log_leak[
    ['serial_number', 'failure_time', 'tag']
]
df_failure_leak['dt_failure_time'] = pd.to_datetime(df_failure_leak['failure_time'])
df_failure_leak = df_failure_leak.sort_values("dt_failure_time")
df_failure_leak = df_failure_leak[~df_failure_leak.dt_failure_time.isnull()]
df_failure_leak = df_failure_leak.drop_duplicates()

df_kernel_log_leak = df_kernel_log_leak[
    [col for col in df_kernel_log_leak if col not in ['failure_time', 'tag']]
]
df_kernel_log_leak = preprocess(df_kernel_log_leak)

df_trainable_leak = get_data_trainable(
    df_address_log_leak, df_mce_log_leak, df_kernel_log_leak, df_failure_leak)

In [ ]:
df_trainable_leak.shape

## 缓存数据

In [ ]:
cache_name = f"cache_{notebook_id}.pickle"
cache_name

In [ ]:
with open(cache_name, 'wb') as f:
    pickle.dump([df_trainable, df_trainable_leak, df_failure_leak], f)

In [ ]:
with open('cache_i-want-to-be-top-5-v5.pickle', 'rb') as f:
    df_trainable, df_trainable_leak, df_failure_leak = pickle.load(f)

# 训练模型

## 记录特征的栏位

In [ ]:
all_feature_columns = get_all_features(df_trainable)
# 会用来训练
all_feature_columns

In [ ]:
df_trainable.columns

In [ ]:
def nice_str(v):
    if v < 10:
        return '0' + str(v)
    return str(v)


df_trainable['collect_time'] = \
    df_trainable.dt_date.apply(str) + ' ' + \
    df_trainable.dt_hour.apply(nice_str) + ':' + \
    df_trainable.dt_minute.apply(nice_str) + ':00'

df_trainable['dt_collect_time'] = pd.to_datetime(
    df_trainable['collect_time']
)


In [ ]:
import datetime
sn_time = df_trainable.groupby(
    'serial_number'
).agg({'dt_collect_time': ['min', 'max']}).reset_index()

sn_time.columns=['serial_number', 'min_dt', 'max_dt']
sn_time['gap_days'] = sn_time.apply(lambda x:(x.max_dt - x.min_dt).days,axis=1)
sn_time.loc[sn_time.gap_days > 21,'min_dt'] = sn_time.loc[sn_time.gap_days>30].max_dt - datetime.timedelta(days=31)
sn_time = sn_time[['serial_number','min_dt','max_dt']]

In [ ]:
df_trainable = df_trainable.merge(sn_time)
df_trainable = df_trainable[df_trainable.dt_collect_time > df_trainable.min_dt]

In [ ]:
df_trainable.shape

## 标准化数据

In [ ]:
X = df_trainable[
    all_feature_columns
].values

y_remain = df_trainable.remain.values
gc.collect()

## 打标签

In [ ]:
y_binary = np.copy(y_remain)

y_binary[y_binary <= hyper_parameters['pos_threshold']] = 1
y_binary[y_binary > hyper_parameters['pos_threshold']] = 0

## 模型训练

In [ ]:
# 我们采用了LightGBM 模型，实测发现最终结果对参数比较鲁棒
# 我们采用了focal loss 函数作为训练目标，focal loss 函数可以较好的处理样本不均衡的问题，在训练中更加注重小样本的学习，
from scipy.misc import derivative
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score

a,g = 0.9833, 2
def focal_loss_lgb(y_true, y_pred):
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    return grad, hess
  
def focal_loss_lgb_eval(y_true, y_pred):
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False


def focal_loss_xgb(y_pred, y_true):
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    print(grad, hess)
    return grad, hess


def f1_score_eval(y_true, y_pred):  
    print(y_true)
    return 'f1_score', f1_score(y_true, (y_pred > 0.5).astype(int)), True


from sklearn.metrics import fbeta_score, make_scorer
def f1_score_xgb_eval(y_pred, y_true):  
    value = f1_score(y_true.get_label(), (y_pred > 0.5).astype(int))
    return 'f1_score', -1 * value




# https://machinelearningmastery.com/cross-validation-for-imbalanced-classification/
kf = sklearn.model_selection.StratifiedKFold(
    n_splits=hyper_parameters['kfold'], shuffle=True, random_state=2019,
)

clf_list = []

gc.collect()

for idx, (train_index, test_index) in enumerate(kf.split(X, y_binary)):
    print(f"kFold: {idx+1}/{hyper_parameters['kfold']}")
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y_binary[train_index], y_binary[test_index]
    print(np.sum(y_train == 0), np.sum(y_train == 1))
    print(np.sum(y_valid == 0), np.sum(y_valid == 1))
    
    clf = LGBMClassifier(
        learning_rate=0.05,
        n_estimators=1000,
        max_depth= 7,
        subsample=0.7,
        colsample_bytree=0.7,
        random_state=2019,
        metric=None,
        objective=focal_loss_lgb,
    )

    clf.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric=[
            focal_loss_lgb_eval,
            f1_score_eval
        ],
        verbose=1,
        early_stopping_rounds=200
    )
    clf_list.append(clf)

## 保存模型

In [ ]:
%%javascript
// 必须手动跑
var kernel = IPython.notebook.kernel;
var _the_name = window.document.getElementById("notebook_name").innerHTML;
kernel.execute(`notebook_id = '${_the_name}'`);


In [ ]:

model_name = f"{notebook_id}_{exp_id[:10]}.pickle"
model_name

In [ ]:
with open(model_name, 'wb') as f:
    pickle.dump(clf_list, f)

## 模型验证

In [ ]:
def predict(clf_list, X):
    all_results = []
    for clf in clf_list:
        if type(clf).__name__ == 'LGBMClassifier':
            all_results.append(clf.predict_proba(X))
        else:
            all_results.append(clf.predict_proba(X)[:,1])
    return np.array(all_results)

In [ ]:
y_pred_in_sample = np.array(np.mean(predict(clf_list, df_trainable[
    all_feature_columns
]), axis=0))

In [ ]:
th_candidates = np.linspace(0.0, 2, 21)
for th in th_candidates:
    print(f"th={th:.3f}, score={f1_score(y_binary, (y_pred_in_sample > th).astype(int)):.5f}")

# Leak验证

## 读取模型

In [ ]:
model_name = "i-want-to-be-top-5-v5_last_month_eyJncmFudW.pickle"
with open(model_name, 'rb') as f:
    saved_clf_list = pickle.load(f)
model_name

## 读取数据

In [ ]:
X_leak = df_trainable_leak[
    all_feature_columns
].values

y_remain_leak = df_trainable_leak.remain.values

## 完成推理

In [ ]:
y_pred_leak = np.array(np.mean(predict(saved_clf_list, X_leak), axis=0))

In [ ]:
y_pred_leak

## 验证F1

In [ ]:
th_candidates = np.linspace(0.0, 2, 21)
y_binary_leak = y_remain_leak.copy()

y_binary_leak[y_binary_leak <= hyper_parameters['pos_threshold']] = 1
y_binary_leak[y_binary_leak > hyper_parameters['pos_threshold']] = 0

for th in th_candidates:
    print(f"th={th:.3f}, score={f1_score(y_binary_leak, (y_pred_leak > th).astype(int)):.5f}")

In [ ]:
df_trainable_valid = df_trainable.copy()

In [ ]:
df_trainable_valid["y_pred"] = np.array(np.mean(predict(clf_list, df_trainable[
    all_feature_columns
]), axis=0))

In [ ]:
df_test = df_trainable_valid[
    df_trainable_valid.dt_date == datetime.date(2019, 5, 1)
]

In [ ]:
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))
np.set_printoptions(suppress=True)

In [ ]:
df_test[
    df_test.y_pred > 0.6
].sort_values(['dt_hour', 'dt_minute', 'serial_number'])[
    ['dt_hour', 'dt_minute', 'serial_number', 'y_pred']
]

In [ ]:
a = df_test[
    df_test.y_pred > 0.6
].sort_values(['dt_hour', 'dt_minute', 'serial_number']).head(1)[all_feature_columns].values[0].tolist()

b = [5688.0, 0.0, 3.944521497919556, 1442.0, 5.636932510460378, 31.775008127485137, 12.0, 0.0, 68256.0, 10332.0, 352.113618026909, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6636.0, 0.0, 4.601941747572815, 1442.0, 6.576421262203774, 43.24931661796588, 14.0, 0.0, 92904.0, 12054.0, 410.79922103139387, 50885536.0, 0.0, 35288.16643550624, 1442.0, 50431.17017302735, 2543302925.020843, 109936.0, 0.0, 5463100081408.0, 92457832.0, 3150112.540618192, 231456.0, 0.0, 160.51040221914008, 1442.0, 288.3300720950177, 83134.23047431812, 1016.0, 0.0, 157030656.0, 436448.0, 17139.31620573003, 2884.0, 2.0, 2.0, 1442.0, 0.0, 0.0, 2.0, 2.0, 5768.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3792.0, 0.0, 2.6296809986130376, 1442.0, 3.7579550069735848, 14.122225834437835, 8.0, 0.0, 30336.0, 6888.0, 234.74241201793936, 2041.0, 0.0, 1.4153952843273232, 1442.0, 2.6415775958036565, 6.977932194651826, 9.0, 0.0, 12951.0, 3869.0, 156.0929210438449, 7584.0, 0.0, 5.259361997226075, 1442.0, 7.5159100139471695, 56.48890333775134, 16.0, 0.0, 121344.0, 13776.0, 469.4848240358787, 4308.0, 0.0, 2.9875173370319, 1442.0, 5.462250690144852, 29.836182601987918, 19.0, 0.0, 55894.0, 8140.0, 323.7684357685289, 11399.0, 0.0, 7.90499306518724, 1442.0, 11.297330212821903, 127.62966993753857, 25.0, 0.0, 274151.0, 20710.0, 705.7237419840712, 6555.0, 0.0, 4.545769764216367, 1442.0, 8.266683597100128, 68.3380576945643, 29.0, 0.0, 128341.0, 12376.0, 490.38964099988897, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 474.0, 0.0, 0.3287101248266297, 1442.0, 0.4697443758716981, 0.22065977866309117, 1.0, 0.0, 474.0, 838.0, 28.948229652260256, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 0.0062413314840499305, 1442.0, 0.07875517294347172, 0.006202377265356139, 1.0, 0.0, 9.0, 18.0, 4.242640687119285, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1442.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
# for _i, (_a, _b) in enumerate(zip(a, b)):
#     if _a == _b:
#         print(all_feature_columns[_i], _a, _b)
for _i, (_a, _b) in enumerate(zip(a, b)):
    if _a != _b:
        print(all_feature_columns[_i], _a, _b)


In [ ]:
df_address_log_2019_5_1 = df_address_log[
    (df_address_log.dt_collect_time.dt.date == datetime.date(2019, 5, 1)) & (
        df_address_log.dt_collect_time.dt.hour == 3) & (
        df_address_log.dt_collect_time.dt.minute < 10)
]

In [ ]:
df_mce_log_2019_5_1 = df_mce_log[
    (df_mce_log.dt_collect_time.dt.date == datetime.date(2019, 5, 1)) & (
        df_mce_log.dt_collect_time.dt.hour == 3) & (
        df_mce_log.dt_collect_time.dt.minute < 10)
]

In [ ]:
df_kernel_log_2019_5_1 = df_kernel_log[
    (df_kernel_log.dt_collect_time.dt.date == datetime.date(2019, 5, 1)) & (
        df_kernel_log.dt_collect_time.dt.hour == 3) & (
        df_kernel_log.dt_collect_time.dt.minute < 10)
]

In [ ]:
df_address_log_2019_5_1.shape[0] + df_mce_log_2019_5_1.shape[0] + df_kernel_log_2019_5_1.shape[0]

In [ ]:
df_2019_5_1 =  pd.concat([
    df_address_log_2019_5_1, df_mce_log_2019_5_1, df_kernel_log_2019_5_1
]).sort_values("dt_collect_time").reset_index(drop=True)

df_2019_5_1['dt_date'] = df_2019_5_1.dt_collect_time.dt.date
df_2019_5_1['dt_hour'] = df_2019_5_1.dt_collect_time.dt.hour
df_2019_5_1['dt_minute'] = df_2019_5_1.dt_collect_time.dt.minute

df_2019_5_1['granularity'] = (
        df_2019_5_1['dt_date'].astype(str) +
        "_" + df_2019_5_1['dt_hour'].astype(str) +
        "_" + (df_2019_5_1['dt_minute'] // 10).astype(str)
)


In [ ]:
# df_2019_5_1
df_2019_5_1[
 df_2019_5_1.serial_number == "server_3988"   
].shape

In [ ]:
tsfresh.__version__

In [ ]:
all_feature_columns

## 验证分数

### 搜索threshold和pti

In [ ]:
th_candidates = [0.5, 0.55, 0.6, 0.65]# np.linspace(0.0, 1, 11)
pti_candidates = [1, 5, 10, 15]

In [ ]:
import warnings
warnings.filterwarnings('ignore')


df_trainable_leak_valid = df_trainable_leak.copy()
df_trainable_leak_valid['pred'] = y_pred_leak
df_trainable_leak_valid['pti'] = 1

def nice_str(v):
    if v < 10:
        return '0' + str(v)
    return str(v)


df_trainable_leak_valid['collect_time'] = \
    df_trainable_leak_valid.dt_date.apply(str) + ' ' + \
    df_trainable_leak_valid.dt_hour.apply(nice_str) + ':' + \
    df_trainable_leak_valid.dt_minute.apply(nice_str) + ':00'

df_trainable_leak_valid['dt_collect_time'] = pd.to_datetime(
    df_trainable_leak_valid['collect_time']
)



In [ ]:
df_trainable_leak_valid

In [ ]:
for th in th_candidates:
    for pti in pti_candidates:

        sub_df = df_trainable_leak_valid[
            df_trainable_leak_valid.pred > th
        ]
        sub_df['pti'] = pti

        sub_df = sub_df.join(
            df_failure_leak.set_index('serial_number')['dt_failure_time'], how='left', on='serial_number'
        )
        sub_df['ati'] = (sub_df['dt_failure_time'] - sub_df['dt_collect_time']) / pd.Timedelta('1min')
        sub_df = sub_df[(sub_df['ati']>=0)|(sub_df['ati'].isna())]


        # 取每个周期第一个
        sub_df = sub_df.sort_values(by=['serial_number', 'dt_collect_time'])
        pre_ser = -1
        init_pre_time = pd.to_datetime('2018-12-01')
        window_time = pd.Timedelta('7D')
        pre_time = init_pre_time
        judge = []
        for sn, cur_time in sub_df[
            ['serial_number', 'dt_collect_time']
        ].values:
            
            if pre_ser != sn:
                pre_time = init_pre_time
                
            if (cur_time-pre_time) < window_time:
                judge.append(0)
            else:
                judge.append(1)
                pre_time = cur_time
                
            pre_ser = sn
            
        judge = np.array(judge)
        sub_df = sub_df[judge==1].reset_index(drop=True)

        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        n_pp = len(sub_df)
        n_pr = len(df_failure_leak)

        n_tpr = 0
        n_tpp = 0
        non_zero_scores = []

        for sn, pti, ati in sub_df[['serial_number', 'pti', 'ati']].values:
            if pd.notna(ati):
                if 0 <= pti < 7 * 24 * 60: # 待确定
                    if pti <= ati:
                        n_tpp += sigmoid(pti/ati)
                        non_zero_scores.append(sigmoid(pti/ati))
                if ati < 7 * 24 * 60:
                    n_tpr += 1

        precision = n_tpp / n_pp
        recall = n_tpr / n_pr
        
        if (precision + recall) == 0:
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)

        print(f"""score:[{f1*100:.3f}] th:[{th:.5f}] pti:[{pti}]
        n_tpp:{n_tpp:.3f}, n_pp: {n_pp}, precision: {precision:.3f}, n_tpr: {n_tpr}, n_pr: {n_pr}, recall: {recall:.3f}, f1: {f1:.3f}""")


In [ ]:
dict_trainable_counts = df_trainable_leak.groupby("serial_number").count()["dt_date"].to_dict()

In [ ]:
dict_trainable_counts

In [ ]:
sns.histplot(
    df_trainable_leak_valid[
        df_trainable_leak_valid.pred > 0.6
    ].serial_number.drop_duplicates().apply(lambda v: dict_trainable_counts[v]).values
)

In [ ]:
df_trainable_leak_valid[
    df_trainable_leak_valid.pred > 0.6
].serial_number.drop_duplicates().apply(lambda v: [v, dict_trainable_counts[v]])

In [ ]:
import datetime
sn_time = df_data_table_original.groupby(
    'serial_number'
).agg({'dt_collect_time':['min','max']}).reset_index()

sn_time.columns=['serial_number','min_dt','max_dt']
sn_time['gap_days'] = sn_time.apply(lambda x:(x.max_dt - x.min_dt).days,axis=1)
sn_time.loc[sn_time.gap_days>30,'min_dt'] = sn_time.loc[sn_time.gap_days>30].max_dt - datetime.timedelta(days=31)
sn_time = sn_time[['serial_number','min_dt','max_dt']]


### filtering only 1 month data
df_data_table_original = df_data_table_original.merge(sn_time)
df_data_table_original = df_data_table_original[df_data_table_original.dt_collect_time > df_data_table_original.min_dt]